In [1]:
stock = 'GOOG'
days_back = 100

In [2]:
import pandas as pd                              # 表格和数据处理
import matplotlib.pyplot as plt                  # 绘图
import seaborn as sns                            # 更多绘图功能
import pandas_datareader.data as webdata

from itertools import product                    # 一些有用的函数
from tqdm import tqdm_notebook

import datetime                                  # 日期函数

import warnings                                  # 勿扰模式
warnings.filterwarnings('ignore')

%matplotlib inline

In [3]:
now = datetime.date.today()
start = now - datetime.timedelta(days=days_back)
# df = data.DataReader(stock, "iex", start=start, end=end)
df = webdata.get_data_yahoo(stock, start=start, end=now)['Adj Close']

In [4]:
df

Date
2018-07-20    1184.910034
2018-07-23    1205.500000
2018-07-24    1248.079956
2018-07-25    1263.699951
2018-07-26    1268.329956
2018-07-27    1238.500000
2018-07-30    1219.739990
2018-07-31    1217.260010
2018-08-01    1220.010010
2018-08-02    1226.150024
2018-08-03    1223.709961
2018-08-06    1224.770020
2018-08-07    1242.219971
2018-08-08    1245.609985
2018-08-09    1249.099976
2018-08-10    1237.609985
2018-08-13    1235.010010
2018-08-14    1242.099976
2018-08-15    1214.380005
2018-08-16    1206.489990
2018-08-17    1200.959961
2018-08-20    1207.770020
2018-08-21    1201.619995
2018-08-22    1207.329956
2018-08-23    1205.380005
2018-08-24    1220.650024
2018-08-27    1241.819946
2018-08-28    1231.150024
2018-08-29    1249.300049
2018-08-30    1239.119995
                 ...     
2018-09-17    1156.050049
2018-09-18    1161.219971
2018-09-19    1171.089966
2018-09-20    1186.869995
2018-09-21    1166.089966
2018-09-24    1173.369995
2018-09-25    1184.650024
2018-09

In [5]:
# 需要预测的5天日期
d = [None] * 5
j = 0

for i in range(0,7):
    if (now + datetime.timedelta(i)).weekday() < 5:
        d[j] = str(now + datetime.timedelta(i))
        j = j + 1

In [6]:
#时序数据上的线性模型
data = pd.DataFrame(df.copy().astype(float))
data.columns = ["y"]

df = pd.DataFrame({'y':[0,0,0,0,0]},
                   index = d)
data = data.append(df)

for i in range(6, 25):
    data["lag_{}".format(i)] = data.y.shift(i)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

y = data.dropna().y # 删除n/a的数据
X = data.dropna().drop(['y'], axis=1)

# 除去需要预测的5天
train_index = int(len(X)) - 5
X_train = X.iloc[:train_index]
y_train = y.iloc[:train_index]

# 机器学习
lr = LinearRegression()
lr.fit(X_train, y_train)
prediction = lr.predict(X.tail(5))
precentage = prediction.mean()
print(prediction.mean())
print(prediction)
# plt.figure(figsize=(15, 7))
# plt.plot(prediction, "g", label="prediction", linewidth=2.0)


1096.3262560692704
[1082.28170129 1106.67448228 1134.61646906 1073.73404313 1084.32458459]


In [7]:
#储存数据到excel
from openpyxl import load_workbook
df = pd.DataFrame(prediction)
df = df.values.T.tolist()
stock_name = [str(stock)]
wb = load_workbook('/Users/BartSu/Desktop/StockRS_Result.xlsx')
ws = wb.worksheets[0]

ws.append(stock_name)

# Append 2 new Rows - Columns A - D
for row_data in df:
    # Append Row Values
    ws.append(row_data)

wb.save("/Users/BartSu/Desktop/StockRS_Result.xlsx")